In [11]:
import os
os.environ["http_proxy"] = "http://127.0.0.1:8889"
os.environ["https_proxy"] = "http://127.0.0.1:8889"

## Step-1 Import

In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

## Step-2 Load data

In [13]:
import pandas as pd

data = pd.read_csv("./datasets/ChnSentiCorp_htl_all.csv")
data.head()

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"


In [14]:
print(len(data))
data = data.dropna()
print(len(data))

7766
7765


## Step-3 Create dataset

In [15]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    
    def __init__(self) -> None:
        super().__init__()
        self.data = pd.read_csv("./datasets/ChnSentiCorp_htl_all.csv")
        self.data = self.data.dropna()
        
    def __getitem__(self, index):
        return self.data.iloc[index]["review"], self.data.iloc[index]["label"]
    
    def __len__(self):
        return len(self.data)



In [16]:
dataset = MyDataset()
for i in range(5):
    print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', 1)
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', 1)
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', 1)
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', 1)
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', 1)


## Step-4 Split dataset

In [17]:
from torch.utils.data import random_split

train_set, valid_set = random_split(dataset, lengths=[0.9, 0.1])
len(train_set), len(valid_set)

(6989, 776)

In [18]:
for i in range(5):
    print(train_set[i])

('3月28日和3月29日在此住宿两晚，118每晚，酒店的房间装修比较差，门松松垮垮，不可反锁（靠一个防全开的别扣），卫生间的浴室部分就是一个铅管垂下头，28日晚上抵达太晚了没有洗漱，29日发现没有热水，向服务员反应后，她的第一反应是：“你把冷热水方向搞反了”，我在此住了2晚居然连冷热水都分不清？然后是：“你要多放一会”她自己放了，也是温的不足以洗浴。鉴于在西宁一天快捷的经验，我请她跟锅炉房说一声把水温提到一点，她说：“我们的师傅都是专业的，知道该做什么”。放了半个小时还是没有热水后，我们决定按照其它前面的网友提示的去旁边的“新华宾馆”（新华书店边），装修比这家好太多，才100一夜。总结：设施差一点没问题，但是服务员一开口就是顾客的错，顾客不够专业，顾客什么都不知道，我们就这样住不住无所谓。所以打1分。宾馆反馈2008年4月24日：首先感谢贵宾客对我宾馆提出宝贵的意见，我们在看到你的意见后非常重视，也立即做了改进。客房磁卡锁自带反锁，安全上请宾客绝对放心，本宾馆素以安全著称,24小时有保安人员、服务人员值班，各楼道有公共区域有闭路监控,宾馆执行军事化管理。北楼部分房间只有沐浴，如果需要盆浴，可以要求我们的工作人员换房间，或者在预订单中注明要带盆浴的房间。由于我们房间数量较多，锅炉房距离少数房间有点远，输水管道也相应较长，因此这些房间需要将管道中的冷水放完才会有热水,本宾馆的热水都是保证最低水温不能低于75度才能向客房输送，但对于这种情况可能是我们的工作人员没有向贵宾客解释清楚，并且在语气上也有不足之处，在此宾馆向贵宾客深表歉意。宾馆已对此事作出严肃处理。这种情况我宾馆一定杜绝再次发生。希望您能下次入住我们酒店时，我们能给你更好的服务，以后还多对宾馆提出宝贵建议，拉萨广场饭店全体员工真诚欢迎您。再次非常感谢您对我宾馆作出的点评。', 0)
('入住的是80公社标准间，很有特色。交通很便利。唯一不太好的是看不到服务员，打电话叫的话要连续几次才有人出现。', 1)
('酒店位置不是很理想，去金元购物中心还是蛮近的，大约10分钟车程，我入住的是448的房间，正赶上卫生部开会，人很多，另外，2个房间之间有一扇门，隔音效果特别不好，半夜时被隔壁的亲热声吵醒，晕', 0)
('宾馆虽然比较老，房间感觉还可以，服务很好，早餐品种少了点。打车起步价到海边。走路15分钟到好又多超市。在

## Step-5 Create DataLoader

In [19]:
import torch

tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")

def collect_func(batch):
    texts, labels = [], []
    for item in batch:
        texts.append(item[0])
        labels.append(item[1])
    inputs = tokenizer(texts, max_length=128, padding="max_length", truncation=True, return_tensors="pt")
    inputs["labels"] = torch.tensor(labels)
    return inputs

In [20]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_set, batch_size=32, shuffle=True, collate_fn=collect_func)
valide_loader = DataLoader(valid_set, batch_size=64, shuffle=False, collate_fn=collect_func)

In [21]:
next(enumerate(train_loader))[1]

{'input_ids': tensor([[ 101, 3634, 6983,  ...,    0,    0,    0],
        [ 101,  679, 7231,  ...,    0,    0,    0],
        [ 101, 2769, 3221,  ...,  966, 1557,  102],
        ...,
        [ 101, 1086, 3613,  ...,    0,    0,    0],
        [ 101, 3221, 2769,  ..., 1072, 8024,  102],
        [ 101, 6983, 2421,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1,
        0, 0, 0, 0, 1, 1, 1, 1])}

## Step-6 Create Model and Optimizer

In [22]:
from torch.optim import AdamW 

model = AutoModelForSequenceClassification.from_pretrained("./download_models/hfl/rbt3")
if torch.cuda.is_available():
    model = model.cuda()

optimizer = AdamW(model.parameters(), lr=2e-5)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./download_models/hfl/rbt3 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Step-7 Train and Evaluate

In [23]:
def evaluate():
    model.eval()
    acc_num = 0
    with torch.inference_mode():
        for batch in valide_loader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            output = model(**batch)
            pred = torch.argmax(output.logits, dim=1)
            acc_num += (pred.long() == batch["labels"].long()).float().sum()
    return acc_num / len(valid_set)
        
def train(epoch=5, log_step=100):
    global_step = 0
    for ep in range(epoch):
        model.train()
        for batch in train_loader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()
            if global_step % log_step == 0:
                print(f"epoch {ep} global step {global_step} loss {output.loss.item()}")
            global_step += 1
        
        acc = evaluate()
        print(f"epoch {ep} acc {acc}")

## Step-8 Train

In [24]:
train()

epoch 0 global step 0 loss 0.7946959733963013
epoch 0 global step 100 loss 0.21117760241031647
epoch 0 global step 200 loss 0.34721940755844116
epoch 0 acc 0.8775773048400879
epoch 1 global step 300 loss 0.18928979337215424
epoch 1 global step 400 loss 0.08193037658929825
epoch 1 acc 0.8853092789649963
epoch 2 global step 500 loss 0.09853857755661011
epoch 2 global step 600 loss 0.1200331449508667
epoch 2 acc 0.8956185579299927
epoch 3 global step 700 loss 0.062062494456768036
epoch 3 global step 800 loss 0.05296177789568901
epoch 3 acc 0.8737112879753113
epoch 4 global step 900 loss 0.07200489938259125
epoch 4 global step 1000 loss 0.08653172105550766
epoch 4 acc 0.8788659572601318



## Step-9 Test

In [25]:
sen = "这家酒店的环境非常好，价格也便宜，值得推荐"

id2label = {0: "neg", 1: "pos"}

with torch.inference_mode():
    inputs = tokenizer(sen, return_tensors="pt")
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}
    output = model(**inputs)
    pred = torch.argmax(output.logits, dim=-1)
    print(id2label[pred.item()])
    

pos


In [26]:
from transformers import pipeline

model.config.id2label = id2label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

pipe(sen)

[{'label': 'pos', 'score': 0.9996730089187622}]

## Step-10 Save and load

In [27]:
model.save_pretrained("./save_models/rbt3")

model = AutoModelForSequenceClassification.from_pretrained("./save_models/rbt3")